In [1]:
# good tutorial for elasticsearch python http://nitin-panwar.github.io/Elasticsearch-tutorial-for-beginners-using-Python/
#https://elasticsearch-py.readthedocs.io/en/master/
#http://snowsyn.net/2016/10/25/having-fun-with-elasticsearch-and-python/
#https://marcobonzanini.com/2015/02/02/how-to-query-elasticsearch-with-python/
#https://www.elastic.co/guide/en/elasticsearch/guide/current/routing-value.html for the report how distributed indexing works
#http://xpo6.com/using-elasticsearch-analyzer-remove-stop-words-from-text/
#https://www.elastic.co/guide/en/elasticsearch/guide/current/data-in-data-out.html  every field has a dedicated inverted index for fast retrieval.
#https://dzone.com/articles/23-useful-elasticsearch-example-queries

from elasticsearch import Elasticsearch 
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [2]:
e1={
    "first_name":"nitin",
    "last_name":"panwar",
    "age": 27,
    "about": "Love to play cricket",
    "interests": ['sports','music'],
}
print(e1)

{'first_name': 'nitin', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}


In [3]:
res = es.index(index='megacorp',doc_type='employee',id=1,body=e1)

In [4]:
# Let's insert some more documents
e2={
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}
e3={
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
}

In [5]:
res=es.index(index='megacorp',doc_type='employee',id=2,body=e2)
res=es.index(index='megacorp',doc_type='employee',id=3,body=e3)

In [6]:
res=es.get(index='megacorp',doc_type='employee',id=3)
print(res)

{'_index': 'megacorp', '_type': 'employee', '_id': '3', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'first_name': 'Douglas', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}}


In [7]:
print(res['_source'])

{'first_name': 'Douglas', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}


In [8]:
#res=es.delete(index='megacorp',doc_type='employee',id=1)
#res=es.delete(index='megacorp',doc_type='employee',id=2)
#res=es.delete(index='megacorp',doc_type='employee',id=3)

#job = {
#    "title": "technology returner programme", "company": "nomura", "description": "nomura diversity critical success harness value perspectives employees different work family experience backgrounds bring firm past experience invaluable us nomura technology division pleased launching first returner programme looking individuals taken career break 18months much longer looking return workplace successful candidates receive significant training support help transition new role world technology fast moving excellent opportunities find fulfilling role also provides flexibility need roles available 4 5 day week basis nomura financial services group integrated global network spanning 30 countries head offices asia connecting markets east west service needs individuals institutions corporates governments four business divisions retail asset management wholesale global markets investment banking merchant banking offer technology returner programme based london office designed experienced professionals ready build previous skills experience want embrace new opportunities ahead paid programme commences april 29th finishes 19th july 2019. end 3-month programme participants invited apply number open roles offer number type roles available outlined programme \u2019 guarantee permanent role ensure developed valuable skills exposure wide-reaching influential network involvement high impact projects global financial services company looking returners join us previous experience business analysts project managers developers specific opportunities offer include strategic projects manager cto office software engineer business analyst team leader security consultant project manager hybrid project manager/it business analyst programme open women men career break least 18 months minimum number 5 years professional experience need able work city based london office duration programme eligible work uk april 2019 july 2019. opportunities available 4 5 days week basis pending preference supported goal support back workplace get taste career global investment bank support line manager coach buddy also employee networks join build network across firm prior 12 weeks provided pre-commencement coaching training day inclusivity one day induction organisation three personal coaching sessions delivered inclusivity \u2019 specialist coach relevant technical training role remuneration benefits opportunity salary aligned market rates also access programme nomura services emergency childcare adult dependent care fitness centre onsite restaurant onsite gp nurse dental practice apply help application process questions hesitate contact us submit c.v. cover letter online tuesday 5th march latest please ensure c.v. shows career break least 18 months immediately preceding date application programme help us get know better include cover letter 've taken break work include motivates want return please place cover letter document cv shortlisted inclusivity touch organise first interview please click 'apply button redirected website", "location": "london", "URL": "https://www.technojobs.co.uk/job/2620662/technology-returner-programme/"}, {"title": "mobile developer in test javascript appium mocha chai", "company": "client server", "description": "mobile developer test javascript appium mocha chai city based financial software house provide range web based trading single-dealer platforms seeking mobile developer test experience testing mobile front end applications six month contract city office mobile developer test responsible developing automated test cases test frameworks new functionality within jenkins environment include functional non-functional testing using variety open source tools libraries 'll collaborate ux designers product owners define implement unit acceptance integration performance tests relaxed city based offices table tennis football casual dress code flexible working requirements *experience testing mobile applications appium *experience javascript testing frameworks including mocha chai *experience working jenkins *experience software tester developer test good appreciation test automation *passionate code quality advocate clean code principles *excellent analysis problem solving skills meticulous attention detail *nice mobile testing experience react react native apply call confidential discussion developer test contract opportunity mobile developer test javascript appium mocha chai rate \u00a3400 \u00a3450 p/day location london city term 6 months start immediate asap", "location": "london", "URL": "https://www.technojobs.co.uk/job/2624214/mobile-developer-in-test-javascript-appium-mocha-chai/"
#    }

#res=es.index(index='technojobs',doc_type='jobs',id=1,body=job)
#print(res)
#es.indices.delete(index='job', ignore=[400, 404])

{'acknowledged': True}

In [9]:
es.indices.create(index='job', body={
   'settings' : {
         'index' : {
              'number_of_shards':5
         },
    "analysis": {
      "filter": {
        "filter_stemmer": {
          "type": "stemmer",
          "language": "english"
        },
            "stopwordEn": {
            "type":       "stop",
            "stopwords":  "_english_"
        }
      },
      "analyzer": {
        "tags_analyzer": {
          "type": "custom",
          "filter": [
            "standard",
            "lowercase",
            "filter_stemmer"
          ],
          "tokenizer": "standard"
        }
      }
    }
   }
})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'job'}

In [10]:
import json
    
with open('all_jobs.json', 'r') as f:
    technojobs_dict = json.load(f)
    #count = 0
    for technojob in technojobs_dict:
        #count+=1
        my_id = technojob.pop('URL', None)
        es.index(index='job', doc_type='job', body=json.dumps(technojob))
        #if count >= 10:
        #    break
    
#for technojob in technojobs_dict:
#    print(technojob['title'])
    



In [12]:
technojobs_dict[:10]

[{'company': 'nhs greater glasgow clyde',
  'date_listed': '13-03-2019',
  'description': "Apply on employer's website                                                                                TO APPLY PLEASE ENSURE YOU ONLY DOWNLOAD, COMPLETE AND SUBMIT THE JOB PACK AVAILABLE FROM Applications are invited for the posts of Clinical Development Fellow in Urology based at Glasgow Royal Infirmary. This post would suit post foundation doctors or  doctors interested in developing their CV. The Clinical Development Fellow is placed within a clinical area and the post will afford successful applicants the opportunity to gain experience. Supervision and training will be provided to enable the Fellow to  develop their clinical skills. The postholder will be expected to participate fully in the work of the department/team and banding will be appropriate to intensity of the rota. The Clinical Development Fellow post will have two in built protected sessions  of professional development time.

In [6]:
def searchFunction(feature, search):
    res = es.search(index="job", doc_type="job", body={"query": {"match": {feature: search}}})
    print("%d documents found" % res['hits']['total'])
    for doc in res['hits']['hits']:
        print("%s) %s" % (doc['_id'], doc['_source'][feature]))
        
def searchFunctionAllFeatures(search):
    res = es.search(index="job", doc_type="job", body={"query": {"multi_match" : {
            "query" : search,
            "fields": ["title^3", "description","location","URL"]#makes the title three times more important
        }}})
    print("%d documents found" % res['hits']['total'])
    for doc in res['hits']['hits']:
        print("%s) %s" % (doc['_id'], doc['_source']))

In [7]:
searchFunction("title","data scientist")

774 documents found
tcA0fWkBX9rBgXuLKY0b) 
            Data Scientist        
IcA0fWkBX9rBgXuLEIeR) 
            Data Scientist         
ZcA0fWkBX9rBgXuLOpJj) 
            Data Scientist         
x8A0fWkBX9rBgXuLWZqq) 
            Data Scientist        
xcA0fWkBX9rBgXuLc6G_) 
            Data Scientist        
5ApZmmkBO93G78ZJoCQU) data scientist
kMA0fWkBX9rBgXuLJYwH) 
            Data Scientist        
o8A0fWkBX9rBgXuLA4Ow) 
            Data Scientist        
YMA0fWkBX9rBgXuLap-5) 
            Data Scientist         
-QpZmmkBO93G78ZJkB8L) data scientist


In [13]:
searchFunctionAllFeatures("hadoop")

113 documents found
w2aSkGkB484-OV2z91VE) {'title': '\n            Hadoop Developer        ', 'company': '\n                        Computer Futures                    ', 'description': ['\n            Start: latest 1st of May 2019', '\r\n', '\r\nEnd: 31st of March 2020 (extension possible)', '\r\n', '\r\nLocation: Stockholm', '\r\n', '\r\nWorkload: 100%', '\r\n', '\r\n ', '\r\n', '\r\nYour tasks mean that you primarily work with the Hadoop platform and SQL server both strategically and operationally to provide the business with data for report tools and reports. Your role includes proposing technical solutions and proactively improving the platform, but also acting as a sounding board for various stakeholders in the business.', '\r\n', '\r\nYour background', '\r\n', '\r\nWe see that you have a solid IT background with an academic degree or equivalent experience acquired in another way. You have at least 3 years of experience with BI solutions. Merit is the experience of Hadoop and the